# Extractor de Características de Audio
Con este algoritmo procesaremos las bases de datos de audios (formato wav.) y pasaremos a la conversión de estos a expectrograma Mel.


##Librerías que se utilizarán

In [ ]:
import math
#librería para funciones matemáticas básicas
import numpy as np
#librería para trabajar con arreglos numéricos y realizar operaciones matemáticas en ellos
import torch
#librería de aprendizaje profundo de PyTorch para trabajar con redes neuronales y tensores
import torch.nn as nn
#módulo de PyTorch para definir arquitecturas de redes neuronales
import torch.nn.functional as F
#módulo de PyTorch con funciones de activación y otras operaciones comunes en redes neuronales

from sklearn.model_selection import KFold
!pip install seaborn
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
import os
import glob
from IPython.display import Audio
import librosa.display
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import librosa
warnings.simplefilter(action='ignore', category=FutureWarning)
from torch.utils.data import TensorDataset
from sklearn.metrics import confusion_matrix #para la función kfold
import seaborn as sn
import pandas as pd
import importlib
import matplotlib_inline
import pickle

#Para cargar la base de datos desde drive
from google.colab import drive
drive.mount('/content/drive')


## Carga de datasets
En este código se procederá a:<br>
<font color=pink><br>-Cargar los datos de audio.<br>
<br>-Obtener las muestras y la frecuencia de muestreo de estas (para el cálculo posterior de los espectrogramas-Mel).<br>
<br>-Mediante la librería Librosa, extraer los espectrogramas Mel de las señales de audio.<br>
<br>-Etiquetar los audios con categorías numéricas asignándoles labels a cada emoción.<br>
<br>-Aumentar los datos mediante la técnica de adición de ruido para balancear los datos de entrada del modelo.<br>
<br>-Guardar los datos procesados.<br>
<br>-Mostrar en un histograma la cantidad de datos por etiquetado.

In [ ]:
#Cargamos en tpath los archivos .wav desde el drive.
tpath = ("/content/drive/MyDrive/Datasets/TESS Toronto emotional speech set data")
data = []
audio = []
labels = []
nombres=[]

name = [x[0] for x in os.walk(tpath)]
name.pop(0)
#Inicializamos la variable donde se guardarán los espectrogramas.
spectrograms = np.zeros((1, 75, 128))


#Recorremos el archivo donde están guardados los audios.
for i,names in enumerate(name):
  print(names)
  file_path = names
  files = os.listdir(file_path)

  for j, filename in enumerate(glob.glob(os.path.join(file_path, '*.wav'))):
    x = filename.split("/")
    y = x[7].split("_")
    nombre = y[1] #Obtenemos los nombres de los audios.
    print(str(nombre))
    nombres.append(nombre)
    z=y[-1].split(".")
    id= z[0] #Se obtiene el identificador de la emoción.
    print("id",id)

    #Obtenemos las muestras y frecuencias de muestreo de los audios.
    sample, sr = librosa.load(filename)
    audio.append(sample)
    duration = len(sample)/sr

    #Se extraen los Mel-espectrogramas de las muestras de audio con el uso de la librería Librosa.
    spec = librosa.feature.melspectrogram(
        y= sample,
        sr=sr,
        n_fft= 1024,
        hop_length= 512,
        n_mels= 128,
        fmax= 8000,
        win_length= 400
    )


    #Hacemos la transpuesta a los espectrogramas y los pasamos a decibelios para proceder con la división en segmentos de estos espectrogramas (longitud=75).
    spec = librosa.power_to_db(spec).transpose()
    print(spec.shape)
    div = (spec.shape[0] // 75)
    print(spec.shape[0])
    size_new = int(div*75)
    spec = spec[:size_new]
    spec = spec.reshape((div, 75, 128))



    #Etiquetamos cada identificador de cada segmento de espectrograma, el cuál dependiendo del id corresponde a una u otra emoción, un valor entero.
    for k in range(div):
      if (id == "Fear") or (id == "fear"):
        lab = int(6)
      elif (id == "Pleasant") or (id == "pleasant") or (id == "ps"):
        lab = int(5)
      elif (id == "Sad") or (id == "sad"):
        lab = int(4)
      elif id == "angry":
        lab = int(3)
      elif id == "disgust":
        lab = int(2)
      elif id == "happy":
        lab = int(1)
      else:
        lab = int(0)
      labels.append(lab)

    spectrograms = np.concatenate((spectrograms, spec))
    print("Label: "+str(lab)+"\tSpectrogram shape: "+str(spec.shape) )
    print()


    #Pasamos al aumento de datos mediante la técnica de adición de ruido.
    if (lab == 3) or (lab == 6):
      y_noise = sample
      rms = math.sqrt(np.mean(y_noise**2))
      noise = np.random.normal(0, rms, y_noise.shape[0])
      y_noise = y_noise + noise
      spec_t = librosa.feature.melspectrogram(
        y= sample,
        sr=sr,
        n_fft= 1024,
        hop_length= 512,
        n_mels= 128,
        fmax= 8000,
        win_length= 400
      )
      spec_t = librosa.power_to_db(spec_t).transpose()
      div = (spec_t.shape[0] // 75)
      size_new = int(div*75)
      spec_t = spec_t[:size_new]
      spec_t = spec_t.reshape((div, 75, 128))
      for k in range(div):
        labels.append(lab)
      spectrograms = np.concatenate((spectrograms, spec_t))


    if (lab == 6):
      y_noise3 = sample
      rms = math.sqrt(np.mean(y_noise3**2))
      noise3 = np.random.normal(0, rms, y_noise3.shape[0])
      y_noise3 = y_noise3 + noise3 + noise3
      spec_t3 = librosa.feature.melspectrogram(
        y= sample,
        sr=sr,
        n_fft= 1024,
        hop_length= 512,
        n_mels= 128,
        fmax= 8000,
        win_length= 400
      )
      spec_t3 = librosa.power_to_db(spec_t3).transpose()
      div = (spec_t3.shape[0] // 75)
      size_new = int(div*75)
      spec_t3 = spec_t3[:size_new]
      spec_t3 = spec_t3.reshape((div, 75, 128))
      for k in range(div):
        labels.append(lab)
      spectrograms = np.concatenate((spectrograms, spec_t3))


#Mostramos el espectrograma.
plt.figure()
spec = spec.reshape((75,128))
librosa.display.specshow(spec)
plt.colorbar()

In [ ]:
#Verifico que el número de labels coincidan con el número de espectrogramas.
spectrograms = spectrograms[1:,:,:] #le resto a la posición 1 de spectrograms 1 valor que le añadí al inicializar la variable con np.ceros
print(spectrograms.shape, "Spectrograms array")
print(len(labels))
labels = np.asarray(labels, dtype=int) #Convierte la lista diferentes en una matriz NumPy con el tipo de datos entero (dtype=int).

"""La función TensorDataset() crea un objeto de conjunto de datos PyTorch que se utiliza comúnmente para la entrada de modelos de aprendizaje
automático de PyTorch. En general, este objeto se utiliza para que los datos puedan ser entregados al modelo en lotes durante el entrenamiento."""
dataset = TensorDataset(
    torch.from_numpy(spectrograms), #La función torch.from_numpy convierte los datos desde arreglos de NumPy a tensores de PyTorch
    torch.from_numpy(labels),
)

"""En resumen, el código crea un objeto TensorDataset que contiene los datos de entrada y salida para un modelo de aprendizaje automático
y los almacena como tensores de PyTorch."""

In [ ]:
#clase utilizada para cargar datos almacenados en formato pickle en Python (forma de serializar y guardar objetos de Python en archivos binarios)

def dataLoaderPickle(filename_train, filename_val, batch):
    """filename_train, que es el nombre del archivo que contiene los datos de entrenamiento filename_val, que es el nombre del archivo que contiene los datos de validación
    y batch que es el tamaño de lote utilizado para cargar los datos en el modelo."""

    #Cargamos los datos de filename_train (datos para entrenar) en dataset_train
    infile = open(filename_train, 'rb')
    dataset_train = pickle.load(infile)
    infile.close()

    #Cargamos los datos de filename_val (datos para validar) en dataset_val
    infile = open(filename_val, 'rb')
    dataset_val = pickle.load(infile)
    infile.close()
    print("Training size: ", len(dataset_train))
    print("Validation size: ",len(dataset_val))

    train_dataloader = torch.utils.data.DataLoader(
        dataset_train,
        batch_size = batch,
        num_workers=2,
        shuffle=True #mezclamos datos de entrenamiento y aumentar la diversidad de las muestras que se utilizan en cada iteración del entrenamiento.
    )
    val_dataloader = torch.utils.data.DataLoader(
        dataset_val,
        batch_size=batch,
        num_workers=2,
        shuffle=True
    )
    return train_dataloader, val_dataloader

In [ ]:
#clase utilizada para guardar datos en un formato específico en Python
def dataSaver(dataset, filename):
    #filename = 'Scripted_PHQ8_Eng'
    outfile = open(filename, 'wb')
    pickle.dump(dataset, outfile)
    outfile.close()

In [ ]:
#Se guarda el conjunto de datos dataset en un archivo llamado "Dataset_Aleman".
ruta_archivo = '/content/drive/MyDrive/Datasets/Dataset_Toronto_ruido_m6'
dataSaver(dataset, ruta_archivo)

In [ ]:
#Para mostrar el histograma donde se muestra la cantidad de datos por emoción
plt.close('all')
n, bins, patches = plt.hist(x=labels, bins='auto', color='#0504aa' , alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Is my data balanced?')
plt.text(23,45,r'$\mu15, b=3$')
maxfreq = n.max()
plt.ylim(ymax=np.ceil(maxfreq/10)*10 if maxfreq % 10 else maxfreq + 10)
plt.show()

# Creación del modelo DL para clasificación de emociones
La arquitectura del modelo de este TFG consiste en combinar 4 redes neuronales diferentes: ResCNN, CNN, LSTM (como RNN) y FC.

##Train

In [ ]:
#Training

def trainXD(epochs, model_mlt, train_loader, optimizer, criterion, iter_meter):
    acc_emotion_list = []
    loss_list = []
    label_emotion_list = []
    predicted_list_emotion = []
    loss_tot_list = []
    accuracy_emotion = []

    #epochs: El número de épocas para entrenar el modelo, cantidad de veces que el modelo recorrerá todo el conjunto de datos de entrenamiento durante el proceso de entrenamiento.
    #model_mlt: El modelo de aprendizaje profundo que se entrena.
    #train_loader: El cargador de datos que proporciona los datos de entrenamiento.
    #optimizer: El optimizador que se utiliza para salir de train cuando llega a un punto sin salida.
    #criterion: La función de pérdida que se utiliza para calcular la pérdida entre las predicciones del modelo y las etiquetas reales.
    #iter_meter: Un medidor que se utiliza para realizar un seguimiento del número de iteraciones a través de los datos de entrenamiento.



    """
    Por cada época pasa la variable de entrada por todo el modelo, este saca una previsión, le pide que calcule la variable pérdida que es el loss
    y luego vuelve para atrás y vuelve a hacerlo todo en función de esa variable
    """

    for epoch in range(epochs):
        model_mlt.train() #notebook
        total_training_loss = 0

        for x,y in train_loader:
            inputs = x
            label_emotion = y.long()
            label_emotion_list.extend(label_emotion.detach().numpy())

            optimizer.zero_grad()
            phq8 = model_mlt(inputs.float())
            phq8 = phq8.squeeze(0)

            predicted_list_emotion.extend((torch.max(torch.exp(F.log_softmax(phq8,dim=1)),1)[1]).detach().numpy())

            loss = criterion_emotion(phq8,label_emotion) #esto hay que cambiarlo al estado de ánimo

            loss.backward()
            optimizer.step()
            iter_meter.step()
            total_training_loss += loss

            loss_list.append(loss.item())
            #Track accuracy
            total_emotion = label_emotion.size(0)
            _, predicted = torch.max(phq8.data,1)
            correct_emotion = (predicted == label_emotion).sum().item()
            acc_emotion_list.append(correct_emotion/total_emotion)

        if epoch == (epochs-1):
          print('Train Epoch: {} \tLoss: {:.4f}\tAccuracy: {:.4f}'.format(
              epoch,
              np.mean(loss_list),
              np.mean(acc_emotion_list)
          ))
        loss_tot_list.append(np.mean(loss_list))
        accuracy_emotion.append(np.mean(acc_emotion_list))


    #Printing Confusion Matrix
    """ Matriz que en el eje x los valores que quiero dar y en el y lo que deberían dar y van sumando """

    classes_emotion = ('0','1','2','3','4','5','6')
    cf_matrix_emotion = confusion_matrix(label_emotion_list, predicted_list_emotion)
    df_cm = pd.DataFrame(cf_matrix_emotion/np.sum(cf_matrix_emotion)*10,index = [i for i in classes_emotion], columns=[i for i in classes_emotion])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix emotionression Training')

    fig, axs = plt.subplots(2)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Training Loss')
    axs[0].set(xlabel= 'Epoch', ylabel='Loss')
    axs[1].plot(range(epochs), accuracy_emotion)
    axs[1].set_title('Training Accuracy')
    axs[1].set(xlabel= 'Epoch', ylabel='Accuracy')
    plt.show()

##Test

In [ ]:
def testXD(epochs, model_mlt, train_loader, optimizer, criterion, iter_meter):
    acc_emotion_list = []
    loss_list = []
    label_emotion_list = []
    predicted_list_emotion = []
    loss_tot_list = []
    accuracy_emotion = []
    epochs = 1
    #Training
    for epoch in range(epochs):
        model_mlt.train()
        total_training_loss = 0

        for x,y in train_loader:
            inputs = x
            label_emotion = y.long()
            label_emotion_list.extend(label_emotion.detach().numpy())

            optimizer.zero_grad()
            phq8 = model_mlt(inputs.float())
            phq8 = phq8.squeeze(0)

            predicted_list_emotion.extend((torch.max(torch.exp(F.log_softmax(phq8,dim=1)),1)[1]).detach().numpy())

            loss = criterion_emotion(phq8,label_emotion)

            '''
            A diferencia del Training, esto está comentado porque lo que no hace es ir para atrás, porque no hace falta
            aquí le doy el resultado por toda la red y que me diga lo que vale, no quiero que me recompute los valores, sino
            que quiero ver que lo que he hecho está bien

            loss.backward()
            optimizer.step()
            iter_meter.step()'''
            total_training_loss += loss

            loss_list.append(loss.item())
            #Track accuracy
            total_emotion = label_emotion.size(0)
            _, predicted = torch.max(phq8.data,1)
            correct_emotion = (predicted == label_emotion).sum().item()
            acc_emotion_list.append(correct_emotion/total_emotion)

        print('Test Epoch: {} \tLoss: {:.4f}\temotionression Accuracy: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.mean(acc_emotion_list)
        ))
        loss_tot_list.append(np.mean(loss_list))
        accuracy_emotion.append(np.mean(acc_emotion_list))


    #Printing Confusion Matrix
    classes_emotion = ('0','1','2','3','4','5','6')
    cf_matrix_emotion = confusion_matrix(label_emotion_list, predicted_list_emotion)
    df_cm = pd.DataFrame(cf_matrix_emotion/np.sum(cf_matrix_emotion)*10,index = [i for i in classes_emotion], columns=[i for i in classes_emotion])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Testing')

    fig, axs = plt.subplots(2)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Test Loss')
    axs[0].set(xlabel= 'Epoch', ylabel='Loss')
    axs[1].plot(range(epochs), accuracy_emotion)
    axs[1].set_title('Test Accuracy')
    axs[1].set(xlabel= 'Epoch', ylabel='Accuracy')
    plt.show()

##Redes neuronales usadas

###ResCNN

In [ ]:
#clase de red neuronal convolucional residual que consta de dos capas convolucionales, una capa de normalización de lotes, una función
#de activación ReLU, una capa de regularización Dropout

class ResCNN(nn.Module):
    """
    Primero declaro la clase ResCNN, una red CNN es una red convolucional
    """
    def __init__(self, in_channels, out_channels, stride=1, downsample= None):

        """
        Le entra el número de canales que tengo 128 (banda frecuencial)
        y qué es lo que quiero que salga (le puedo dar el valor que quiera pero en este caso será 32)
        """
        super(ResCNN, self).__init__()

        #BatchNorm1d --> La normalización de lotes se aplica típicamente después de la salida de una capa convolucional
        #Conv1d --> Convolución de 1D
        #RELU --> Función de activación
        self.conv1 = nn.Sequential(
            nn.BatchNorm1d(in_channels),
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(out_channels, out_channels, kernel_size=1)
        )

        #downsample --> reducción de la tasa de muestreo, es decir, disminución de la resolución de una señal o conjunto de datos
        #(implica tomar muestras en intervalos menos frecuentes o eliminar ciertas muestras para reducir la cantidad de información)
        self.downsample = downsample

        self.relu = nn.ReLU()

        self.out_channels = out_channels #almacena el número de canales de salida de la capa convolucional.

        self.dropout = nn.Dropout(p=0.2)
        """ técnica de regularización para prevenir el sobreajuste. Durante el entrenamiento, se desactivan aleatoriamente el 20%  de las unidades de la capa de entrada impidiendo
        así que las unidades de la red dependan demasiado de las demás y, en última instancia, promueve un aprendizaje más robusto y generalizable.
        """

    def forward(self, x):
        residual = x
        output = self.conv1(x)
        output = self.conv2(output)
        if self.downsample:
            residual = self.downsample(x)
        output += residual
        output = self.relu(output)
        #output = self.dropout(output)
        return output

    """El código implementa una operación de residual connection en una red neuronal convolucional, donde se suma la entrada original con la salida de las capas convolucionales.
    Luego, se aplica una función de activación ReLU al resultado y se devuelve como salida."""

###LSTM

In [ ]:
"""El código define una capa LSTM (Long Short-Term Memory) en una red neuronal recurrente (RNN) con ciertas dimensiones y capas.
En el método forward, se pasa la entrada a través de la capa LSTM y se devuelve el resultado."""

class LSTM(nn.Module):

    def __init__(self, n_rnn_dim, h_rnn_layer, n_rnn_layers):
        super(LSTM,self).__init__()
        self.lstm = nn.LSTM(
            input_size= n_rnn_dim,
            hidden_size= h_rnn_layer,
            num_layers= n_rnn_layers,
            batch_first=True,
            bidirectional=True
        )

    def forward(self,x):
        x = self.lstm(x)
        return x

###FC

In [ ]:
""" Esta función define la red FC, la cual realiza una transformación lineal de la entrada, aplica una función de activación para introducir no linealidad
y, opcionalmente, normaliza los valores por lotes para mejorar el rendimiento de la red neuronal."""

class FullyConnected(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(FullyConnected, self).__init__()
        self.fc = nn.Linear(in_features=in_channels, out_features=out_channels)
        self.activation = nn.ReLU()
        self.bn = nn.BatchNorm1d(in_channels)

    def forward(self, x):
        #x = self.bn(x)
        x = self.fc(x)
        x = self.activation(x)
        return x


##Arquitectura del modelo DeepSpeech

In [ ]:
class DeepSpeech(nn.Module):#my deepspe.. puede usar las funciones de nn.Module
    """Donde creo mi modelo, junto todas estas redes neuronales definidas anteriormente"""


    def __init__(self, n_cnn_layers, n_rnn_dim, h_rnn_layer, n_rnn_layers, n_classes, stride=2, dropout=0.1):
        super(DeepSpeech, self).__init__() #aquí le indico a DeppSpeech que nn.Module es su madre y puedo usar sus funciones

        self.dense = nn.Conv1d(75, 32, 1) #mi danse va a tomar el valor que devuelva la función nn.Conv1d
        """De la primera red que tengo (convolución) quiero que me pase de los 75 que tengo de entrada a 32 donde el size del filtro
        kernel=1 """
        """Estos tensores de salida representarán las características aprendidas por los filtros en cada canal de salida y se utilizarán como
        entrada para las capas subsiguientes de la red neuronal para realizar tareas como clasificación"""

        self.cnn = nn.Sequential(*[
            ResCNN(
                32,
                32
            )
            for i in range(n_cnn_layers)
        ])

        #nn.linear(a,b,TRUE), a-->tamaño muestra de entrada,  b--> tamaño muestra de salida
        #se encuentra en la fase de procesamiento final de la red neuronal, donde se combinan las características extraídas para generar la salida deseada.
        ## n_rnn_dim quiero que la dimensión de LSTM sea de 256 muestras

        self.dense2 = nn.Linear(
            32*128,
            n_rnn_dim #256
        )

        """ Después de esto quiero que me haga un filtro lineal y me pase de estos 32*128 (estos 1128 porque antes tenía una matriz de
        128*75 y ahora la tengo de 128*32 --> álgebra lineal) que tengo."""


        self.lstm = nn.LSTM(
            input_size= n_rnn_dim,
            hidden_size= h_rnn_layer,
            num_layers= n_rnn_layers,
            batch_first=True,
            bidirectional=True,
        )

        """Del LSTM sale lo mismo, los 128*32 y esto sigue siendo más que el número de clases por lo que paso al FullyConnected, bajando así el número de clases"""
        #Bajo la matriz al número de clases
        self.fc = FullyConnected(
            in_channels= n_rnn_dim,
            out_channels= n_classes
        )

    #Se definen muchos transpose para trasponer ya que cada ves que se hace cambio de datos hay que hacerlo
    def forward(self, x):
        batch,freq,width = x.shape
        x = self.dense(x)
        x = self.cnn(x)
        x = x.view(x.size(0),x.size(1)*x.size(2))
        #x = x.transpose(0,1)
        x = self.dense2(x)
        #x = x.transpose(0,1)
        x,_ = self.lstm(x)
        #x = self.atten(x)
        x = x.transpose(0,1)
        #x = x[:,-1]
        x = x.transpose(0,1)
        x = self.fc(x)
        return x

##Optimizadores

In [1]:
"""Aquí se define un optimizador que se llama AdamW que es un tipo de algoritmo que existe en internet, que cuando entro a un callejón de salida cómo sale?, sirve para evitar estos puntos"""

import torch.optim as optim

def optimizerNet(model, hparams, train_loader):
    optimizer = optim.AdamW(
        model.parameters(),
        hparams['learning_rate']
    )
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=hparams['learning_rate'],
        steps_per_epoch= int(len(train_loader)),
        epochs= hparams['epochs'],
        anneal_strategy= 'linear'
    )
    return optimizer, scheduler


Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f7dc941bf40>
Traceback (most recent call last):
  File "/usr/lib/python3.10/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [ ]:
"""La clase "IterMeter" es utilizada para realizar un seguimiento del progreso de las iteraciones. Inicialmente, se establece el valor en 0 y luego
se incrementa en 1 con cada llamada a la función "step". La función "get" devuelve el valor actual almacenado en "val"."""


class IterMeter(object):
    """Sirve para que vaya avanzando"""
    #Keeping track of iterations
    def __init__(self):
       self.val = 0 #inicializo

    def step(self):
        self.val += 1 #le va sumando 1 a 1 a val

    def get(self):
        return self.val #devuelve la posición de val actual

##KFold + Hiperparámetros

In [ ]:
"""Aquí defino mis hyperparámetros, estos son una serie de números que tengo que ir optimizando para ver cuál es mejor para mi modelo"""


#Constantes

learning_Rate = 0.0005
"""lo mucho que le dejo aprender, lo mucho que le dejo que se acerque a este callejón sin salida (entre 0-1), cuánto más grande
sea el rate más fácil es que no llegue al callejón de salida pero menos aprende """

batch_size = 128
"""El tamaño de cuántas muestras entran, cuántas más muestras entren más aprende pero más lento va"""

epochs = 300 #El número de veces que entreno

"""KFold es que cojo el dataset y hago reparticiones aleatrorias, 5 particiones (porque la K=5) distintas, para asegurar que justamente no haya
entrenado con unos datos claves que no funcionan o con una convinación clave que funciona
"""
k=5 #entreno 90% y validación 10
splits=KFold(n_splits=k,shuffle=True,random_state=42)
#shuffle=True mezcla todo
#random_state = que tenga un ángulo 42 de libertad


foldperf={}

"""Es lo que le tengo que mandar al modelo para que lo números de antes, intchanell, outchanell aquí es donde lo declaro"""
hparams = {
    "n_cnn_layers": 6, #quiero que me haga 6 layers de CNN
    "n_rnn_layers": 1, #quiero que me haga sólo 1 vez la de LSTM
    "rnn_dim": 256, #quiero que la dimensión de LSTM sea de 256 muestras
    "h_rnn_layers": 128, #me lo mires 128 veces--> vector clave respecto a los 256 que tenía antes
    "n_class": 7, #quiero que hayan 7 clases (en este caso, en otros datasets las que hayan)
    "n_feats": 64,
    "stride": 2,
    "dropout": 0.3,
    "learning_rate": learning_Rate, #0.0005
    "batch_size": batch_size, #128 muestras
    "epochs": epochs #300
}


#Optimizing Model
weights = [1.0,1.0,1.0,1.0,1.0,1.0,1.0]
"""Con los weights penalizo las clases más pobladas
Lo que hace es que donde tenga menos datos puedo hacer que cuando calcule la probabilidad de esta sea más baja, penalizar donde tenga más datos
"""
class_weights = torch.FloatTensor(weights)

criterion_emotion = nn.CrossEntropyLoss(weight=class_weights) #.to(device)
"""Esta es la función con la que voy a evaluar que funciona, hay muchos tipos de funciones de pérdidas (MCE...), en este caso hacemos la entropía
cruzada
"""
criterion_anx = nn.CrossEntropyLoss()
iter_meter = IterMeter()

##Entrenamiento del modelo

In [ ]:
#Aquí entreno 5 veces el modelo, con 5 tipos de datos distintos
#En cada iteración, el tamaño de train_idx será 4/5 del tamaño total de los datos y el tamaño de val_idx será 1/5

ruta_archivo = '/content/drive/MyDrive/Datasets/Dataset_Toronto_ruido_m6'
infile = open(ruta_archivo, 'rb')
Dataset_Toronto_ruido_m6 = pickle.load(infile)
infile.close()

for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(Dataset_Toronto_ruido_m6)))): #90% entreno y 10% testing
    print('Fold {}'.format(fold+1)) #En total habrán 5 Folds que serán los difernetes conjuntos de datasets que entrene.

   #pasamos las variables a tipo sampler
    train_sampler = SubsetRandomSampler(train_idx) #clase importada de la librería torch
    test_sampler = SubsetRandomSampler(val_idx)
    """Con el SubsetRandomSampler, cogeríamos unos x valores randoms del total del dataset train_idx, como después de train_idx no tiene ningún
    valor, pone por predeterminado el NONE, es decir, que pilla todos los datos de train_idx por lo que en realidad estamos pasando de un tipo
    de variable a varialbe tipo sampler"""

    #Cargamos el dataset de 128 en 128 de forma random (definida en train_sampler con la función SubsetRandomSampler)
    train_loaderUns = DataLoader(Dataset_Toronto_ruido_m6, batch_size=batch_size, sampler=train_sampler)
    test_loaderUns = DataLoader(Dataset_Toronto_ruido_m6, batch_size=batch_size, sampler=test_sampler)

    #Llama al init de la clase DeepSpeech
    model = DeepSpeech(
        hparams['n_cnn_layers'],
        hparams['rnn_dim'],
        hparams['h_rnn_layers'],
        hparams['n_rnn_layers'],
        hparams['n_class'],
        hparams['stride'],
        hparams['dropout']
    )

    optimizer, scheduler = optimizerNet(model, hparams, train_loaderUns)

    trainXD(epochs, model, train_loaderUns, optimizer, criterion_emotion, iter_meter)
    testXD(epochs, model, test_loaderUns, optimizer, criterion_anx, iter_meter)



#Algoritmo de almacenamiento del modelo

In [ ]:
ruta_modelo = '/content/drive/MyDrive/Datasets/modelo_entrenado_TORONTO.pth'
torch.save(model.state_dict(), ruta_modelo)


In [ ]:
ruta_modelo = '/content/drive/MyDrive/Datasets/modelo_entrenado_TORONTO.pth'
modelo_cargado = DeepSpeech(...)
modelo_cargado.load_state_dict(torch.load(ruta_modelo))
modelo_cargado.eval()